In [1]:
%load_ext autoreload
%autoreload 2

In [9]:
import pandas as pd
import geojson
import numpy as np

# Local imports
import preproc
import calc

## Get the data!

In [3]:
rooftops = preproc.get_rooftops("../data/rooftop/area-azi-tilt-csv-7.csv")

In [4]:
zip_from_coords = preproc.create_zip_from_coords_getter("../data/us-zip-code-latitude-and-longitude.csv")

## Main process

In [6]:
# Read the solar data
with open('../data/solardata.json') as f:
    solar_data = geojson.load(f)

In [7]:
months = ["JANUARY", "FEBRUARY", "MARCH", "APRIL", "MAY", "JUNE" , "JULY", "AUGUST", "SEPTEMBER", "OCTOBER", "NOVEMBER", "DECEMBER", "ANNUAL"]

In [10]:
for data in solar_data.features:
    props = data["properties"]
    x = props["X_COORD"]
    y = props["Y_COORD"]
    zipcode = zip_from_coords(long=x, lat=y)
    props["zip"] = zipcode

    for m in months:
        solar = props[m]
        if zipcode in rooftops["zip"]:
            roofs_at_zip = rooftops.loc[rooftops["zip"] == 1001].iloc[0]
            abs_pot = calc.calc_for_solar(roofs_at_zip, solar)
            if isinstance(abs_pot, np.ndarray):
                abs_pot = abs_pot.mean()

            norm_pot = calc.calc_for_solar(roofs_at_zip, solar, normalize=True)
            if isinstance(norm_pot, np.ndarray):
                norm_pot = norm_pot.mean()

            props[f"abs_pot_{m}"] = abs_pot
            props[f"norm_pot_{m}"] = norm_pot
        else:
            props[f"abs_pot_{m}"] = 0
            props[f"norm_pot_{m}"] = 0

In [11]:
# Dump the result
with open("result.json", "w") as f:
    geojson.dump(solar_data, f)